In [1]:
!pip install torch torchvision torchaudio
!pip install datasets transformers
!pip install g2p_en praatio librosa

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
from os.path import exists, join, expanduser

os.chdir(expanduser("~"))
charsiu_dir = 'charsiu'
if exists(charsiu_dir):
  !rm -rf /root/charsiu
if not exists(charsiu_dir):
  ! git clone -b development https://github.com/lingjzhu/$charsiu_dir
  ! cd charsiu && git checkout && cd -

os.chdir(charsiu_dir)

Cloning into 'charsiu'...
remote: Enumerating objects: 686, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 686 (delta 110), reused 79 (delta 79), pack-reused 555
Receiving objects: 100% (686/686), 1.20 MiB | 630.00 KiB/s, done.
Resolving deltas: 100% (364/364), done.
Your branch is up to date with 'origin/development'.
/home/tomi


In [3]:
pip install g2pM

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
import sys
import torch
from datasets import load_dataset
import matplotlib.pyplot as plt
sys.path.append('src/')
#sys.path.insert(0,'src')
from Charsiu import charsiu_forced_aligner, charsiu_attention_aligner

In [7]:
timit = load_dataset('timit_asr', data_dir='/home/tomi/Documents/tesis_speechRate/timit')


/home/tomi/.local/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for timit_asr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/timit_asr
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [8]:
# load data
sample = timit['train'][0]
text = sample['text']
audio_path = sample['file']
print('Text transcription:%s'%(text))
print('Audio path: %s'%audio_path)

Text transcription:She had your dark suit in greasy wash water all year.
Audio path: /home/tomi/Documents/tesis_speechRate/timit/data/TRAIN/DR1/FCJF0/SA1.WAV


In [9]:
# initialize model
charsiu = charsiu_forced_aligner(aligner='charsiu/en_w2v2_fc_10ms')

/home/tomi/.local/lib/python3.10/site-packages/transformers/configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
/home/tomi/.local/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at charsiu/en_w2v2_fc_10ms were not used when initializing Wav2Vec2ForFrameClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.

Forced alignment with a neural forced alignment model

In [10]:
alignment = charsiu.align(audio=audio_path,text=text)

In [11]:
print(alignment)
print('\n Ground Truth \n')
print([(s/16000,e/16000,p) for s,e,p in zip(sample['phonetic_detail']['start'],sample['phonetic_detail']['stop'],sample['phonetic_detail']['utterance'])])

([(0.0, 0.16, '[SIL]'), (0.16, 0.28, 'SH'), (0.28, 0.36, 'IY'), (0.36, 0.44, 'HH'), (0.44, 0.54, 'AE'), (0.54, 0.62, 'D'), (0.62, 0.67, 'Y'), (0.67, 0.68, 'AO'), (0.68, 0.71, 'R'), (0.71, 0.83, 'D'), (0.83, 0.89, 'AA'), (0.89, 0.9, 'R'), (0.9, 1.01, 'K'), (1.01, 1.13, 'S'), (1.13, 1.27, 'UW'), (1.27, 1.29, 'T'), (1.29, 1.3, 'IH'), (1.3, 1.39, 'N'), (1.39, 1.44, 'G'), (1.44, 1.52, 'R'), (1.52, 1.59, 'IY'), (1.59, 1.69, 'S'), (1.69, 1.74, 'IY'), (1.74, 1.87, 'W'), (1.87, 1.95, 'AA'), (1.95, 2.07, 'SH'), (2.07, 2.12, 'W'), (2.12, 2.22, 'AO'), (2.22, 2.25, 'T'), (2.25, 2.34, 'ER'), (2.34, 2.43, 'AO'), (2.43, 2.51, 'L'), (2.51, 2.62, 'Y'), (2.62, 2.71, 'IH'), (2.71, 2.83, 'R'), (2.83, 2.9, '[SIL]')], [(0.0, 0.16, '[SIL]'), (0.16, 0.36, 'she'), (0.36, 0.62, 'had'), (0.62, 0.71, 'your'), (0.71, 1.01, 'dark'), (1.01, 1.29, 'suit'), (1.29, 1.39, 'in'), (1.39, 1.74, 'greasy'), (1.74, 2.07, 'wash'), (2.07, 2.34, 'water'), (2.34, 2.51, 'all'), (2.51, 2.83, 'year'), (2.83, 2.9, '[SIL]')])

 Ground 

In [ ]:
plt.plot

In [12]:
# save alignment
charsiu.serve(audio=audio_path,text=text,save_to='./local/sample.TextGrid')

Alignment output has been saved to ./local/sample.TextGrid


Forced Alignment with An Attention Alignment Model

In [13]:
# load data
sample = timit['train'][0]
text = sample['text']
audio_path = sample['file']
print('Text transcription:%s'%(text))
print('Audio path: %s'%audio_path)

Text transcription:She had your dark suit in greasy wash water all year.
Audio path: /home/tomi/Documents/tesis_speechRate/timit/data/TRAIN/DR1/FCJF0/SA1.WAV


In [14]:
# intialize model
charsiu = charsiu_attention_aligner('charsiu/en_w2v2_fs_10ms')

config.json:   0%|          | 0.00/2.62k [00:00<?, ?B/s]

/home/tomi/.local/lib/python3.10/site-packages/transformers/configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/406M [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/charsiu/en_w2v2_fs_10ms/518bda3a5ada591b256cab67161245bbada1b9b4d0d8d95c3006ef9b0787422f?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1713880098&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMzg4MDA5OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9jaGFyc2l1L2VuX3cydjJfZnNfMTBtcy81MThiZGEzYTVhZGE1OTFiMjU2Y2FiNjcxNjEyNDViYmFkYTFiOWI0ZDBkOGQ5NWMzMDA2ZWY5YjA3ODc0MjJmP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiZyZXNwb25zZS1jb250ZW50LXR5cGU9KiJ9XX0_&Signature=hWs5vwopjg2r1OHrner9i8TfhETsUTuBTnDv0%7Eo5sYCTz5ZFqN%7Es0zMDbiNplx6FBmp0rQsyW2i80-MfSZPE4kTyTXRrt8Xzg7WdBuef-Rw0t5YbZs8ntJHNZsgGl2Q6TWOdVIcwDCRtk0xGxenhdxzW8cyVjHYS1jMZGb1V2xilbJBfx6moOpQ5lj5XJD2mtG27TLxZsuobpWut74KuU%7E2BOsi52OQNklLwYztnl4JRaP2g38jeWm0cXVwutwrkJ2dxFSycOzI5u%7E0PEucIS-qV

pytorch_model.bin:  26%|##5       | 105M/406M [00:00<?, ?B/s]

Some weights of the model checkpoint at charsiu/en_w2v2_fs_10ms were not used when initializing Wav2Vec2ForAttentionAlignment: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForAttentionAlignment from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForAttentionAlignment from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForAttentionAlignment were not initialized from the model checkpoint at charsiu/en_w2v2_fs_10ms and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weig

In [15]:
alignment = charsiu.align(audio=audio_path,text=text)

In [16]:
print(alignment)
print('\n Ground Truth \n')
print([(s/16000,e/16000,p) for s,e,p in zip(sample['phonetic_detail']['start'],sample['phonetic_detail']['stop'],sample['phonetic_detail']['utterance'])])

[(0.0, 0.17, '[SIL]'), (0.17, 0.29, 'SH'), (0.29, 0.36, 'IY'), (0.36, 0.44, 'HH'), (0.44, 0.53, 'AE'), (0.53, 0.61, 'D'), (0.61, 0.66, 'Y'), (0.66, 0.7, 'AO'), (0.7, 0.74, 'R'), (0.74, 0.81, 'D'), (0.81, 0.89, 'AA'), (0.89, 0.93, 'R'), (0.93, 1.01, 'K'), (1.01, 1.13, 'S'), (1.13, 1.25, 'UW'), (1.25, 1.29, 'T'), (1.29, 1.34, 'IH'), (1.34, 1.41, 'N'), (1.41, 1.46, 'G'), (1.46, 1.53, 'R'), (1.53, 1.6, 'IY'), (1.6, 1.69, 'S'), (1.69, 1.78, 'IY'), (1.78, 1.88, 'W'), (1.88, 1.99, 'AA'), (1.99, 2.08, 'SH'), (2.08, 2.09, 'ER'), (2.09, 2.15, 'W'), (2.15, 2.23, 'AO'), (2.23, 2.28, 'T'), (2.28, 2.34, 'ER'), (2.34, 2.45, 'AO'), (2.45, 2.53, 'L'), (2.53, 2.64, 'Y'), (2.64, 2.72, 'IH'), (2.72, 2.8, 'R'), (2.8, 2.9, '[SIL]')]

 Ground Truth 

[(0.0, 0.190625, 'h#'), (0.190625, 0.2849375, 'sh'), (0.2849375, 0.3576875, 'ix'), (0.3576875, 0.415125, 'hv'), (0.415125, 0.54825, 'eh'), (0.54825, 0.574375, 'dcl'), (0.574375, 0.6460625, 'jh'), (0.6460625, 0.7198125, 'ih'), (0.7198125, 0.78125, 'dcl'), (0.7812

In [17]:
charsiu.serve(audio=audio_path,text=text,save_to='./local/sample.TextGrid')

Alignment output has been saved to ./local/sample.TextGrid
